In [ ]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [ ]:
%use kandy
%use dataframe


In [ ]:
fun createPlotableDateFrame(datas: List<List<Double>>, labels: List<String>):DataFrame<*>{
    val firstSize = datas[0].size
    require(datas.size == labels.size )
    require(datas.all { it.size == firstSize })
    
    return dataFrameOf(
        "datas" to datas.flatten(),
        "labels" to datas.zip(labels).map {(individualData, label_string) -> List(individualData.size){label_string} }.flatten(),
        "index" to datas.map { sublist -> List(sublist.size){it}}.flatten()
    )
}

//Test
//createPlotableDateFrame(
//    datas = listOf(
//        listOf(1.0,2.0,3.0),
//        listOf(4.0,5.0,6.0),
//        listOf(7.0,8.0,9.0),
//    ),
//    labels = listOf("first label", "second label", "third label")
//).let { 
//    print(it)
//}



In [ ]:
import kotlin.random.Random
import org.kalman.calibration.measurement.Linear2DMeasurementCalibration
import org.kalman.calibration.measurement.MeasurementCalibration

/**
 * Creating 2d linear mode for model Calibration 
 */
val linearCalibration: MeasurementCalibration = Linear2DMeasurementCalibration()
linearCalibration.iterate(100)

val plotData = createPlotableDateFrame(
    datas = listOf(
        linearCalibration.hiddenStates.map { it.maxValue },
        linearCalibration.measuredStates.map { it.maxValue }
    ),
    labels = listOf("TrueHiddeStates", "MeasuredStates")
)


plotData.plot {
    line {
        x("index")
        y("datas")
        color("labels"){
            scale = categorical("TrueHiddeStates" to Color.BLUE, "MeasuredStates" to Color.RED )
        }
    }
    layout.size = 1500 to 450
    
    layout.title = "Generate 2D Linear Data For Calibration"
}


In [32]:
import org.apache.commons.math3.linear.ArrayRealVector
import org.apache.commons.math3.linear.Array2DRowRealMatrix
import org.apache.commons.math3.linear.RealMatrix
import org.apache.commons.math3.linear.RealVector
import org.kalman.wrapper.KalmanFilterProcess

/**
 * Creating Kalman filter and calibrate it with generated dataset.
 */
val kfProcess = KalmanFilterProcess(
    debugEnabled = true,
    states = ArrayRealVector(
        doubleArrayOf(
            50.0 //initial estimation
        )
    ),
    stateTransitionMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(1.0) // assuming the state is still
        )
    ),
    covarianceMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(200.0 * 200.0) // initial co-variance
        )
    ),
    processMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(400.0*400) // assuming process variance of 25
        )
    )
)


repeat(linearCalibration.hiddenStates.size){index ->
    kfProcess.predict()

    kfProcess.update(
        y = linearCalibration.measuredStates[index],
        r = linearCalibration.measurementCovariances[index]
    )
}


/**
 * plot graph 
 */

val test = createPlotableDateFrame(
    datas = listOf(
        linearCalibration.measuredStates.map { it.maxValue },
        linearCalibration.hiddenStates.map { it.maxValue },
        kfProcess.debug().map { it.y.maxValue },
        kfProcess.debug().map { it.x.maxValue }
    ),
    labels = listOf(
        "Calibration_Measurement",
        "Calibration_Hidden_State", 
        "KF_Measurement",
        "KF_Estimate"
        )
)

test.plot {
    line {
        x("index")
        y("datas")
        color("labels"){
            scale = categorical(
                "Calibration_Measurement" to Color.BLUE, 
                "Calibration_Hidden_State" to Color.RED,
                "KF_Measurement" to Color.ORANGE,
                "KF_Estimate" to Color.PURPLE,
                )
        }
    }
    layout.size = 1500 to 450

    layout.title = "Generate 2D Linear Data For Calibration"
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="PCYpc4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Generate 2D Linear Data For Calibration"
},
"mapping":{
},
"data":{
"datas":[252.69281758185298,-2195.346540559586,-1862.4377567195297,-1764.037090250296,-709.7865949525822,-3100.823861869681,751.9905303474352,-774.266816185559,-2858.8515899452295,-2679.1886877370075,3567.9915747390605,6166.6417586993175,-1556.080343785481,1903.9875991283352,4679.6306748409515,3979.0283589817955,6088.336127886674,3314.301624336823,-560.0112830916637,1000.9556506168124,4676.305300031481,7319.409151752767,4664.086754569435,290.0430686601858,7025.618706935469,153.14015140915808,4477.443041001511,5393.194285174017,1435.568913019356,1948.4793712210985,4820.029516862704,8826.573054901955,1867.8467677456115,2976.706801112528,5281.548704068176,6023.102525726356,3715.617042278906,3967.928994977252,4797.25911474523,624.3945855968032,7231.259392035446,3340.970928092926,7851.407816108862,4811.3678535195795,4959.268941957999,6799.9783243844695,7528.718259963739,11284.371959582386,7933.195588043383,6689.617925695694,10649.260020455089,7176.299252610114,2842.336245615371,11078.552472554618,2615.6685504587185,9797.853436032081,6730.823230571008,7393.927450123412,4582.765770388243,4612.956685796911,6637.628348002614,13452.709684510573,10536.653007747671,13208.704659803401,6947.687394466784,13433.616989743019,10644.250513926072,12897.962927605517,13996.592791917394,8901.109445069205,9140.667034756367,6769.083906068028,11595.927157141177,12262.634954762589,5802.816713799841,9080.359944405904,14384.165457808314,7498.687433111909,11284.73094880239,13248.988328014555,14365.671987536698,7522.390674824051,6774.919898871318,10341.630434148274,14054.083025395415,14002.350763903729,16497.89641396652,16047.61142172031,15701.53541663758,11828.54457751544,13003.288310574537,12785.783646893797,12670.45165242047,15298.208905639342,16611.1039903178,14755.5578328428,9022.644061242741,10515.875891044825,16355.99577640969,10432.946374148662,50.0,190.0,330.0,470.0,610.0,750.0,890.0,1030.0,1170.0,1310.0,1450.0,1590.0,1730.0,1870.0,2010.0,2150.0,2290.0,2430.0,2570.0,2710.0,2850.0,2990.0,3130.0,3270.0,3410.0,3550.0,3690.0,3830.0,3970.0,4110.0,4250.0,4390.0,4530.0,4670.0,4810.0,4950.0,5090.0,5230.0,5370.0,5510.0,5650.0,5790.0,5930.0,6070.0,6210.0,6350.0,6490.0,6630.0,6770.0,6910.0,7050.0,7190.0,7330.0,7470.0,7610.0,7750.0,7890.0,8030.0,8170.0,8310.0,8450.0,8590.0,8730.0,8870.0,9010.0,9150.0,9290.0,9430.0,9570.0,9710.0,9850.0,9990.0,10130.0,10270.0,10410.0,10550.0,10690.0,10830.0,10970.0,11110.0,11250.0,11390.0,11530.0,11670.0,11810.0,11950.0,12090.0,12230.0,12370.0,12510.0,12650.0,12790.0,12930.0,13070.0,13210.0,13350.0,13490.0,13630.0,13770.0,13910.0,252.69281758185298,-2195.346540559586,-1862.4377567195297,-1764.037090250296,-709.7865949525822,-3100.823861869681,751.9905303474352,-774.266816185559,-2858.8515899452295,-2679.1886877370075,3567.9915747390605,6166.6417586993175,-1556.080343785481,1903.9875991283352,4679.6306748409515,3979.0283589817955,6088.336127886674,3314.301624336823,-560.0112830916637,1000.9556506168124,4676.305300031481,7319.409151752767,4664.086754569435,290.0430686601858,7025.618706935469,153.14015140915808,4477.443041001511,5393.194285174017,1435.568913019356,1948.4793712210985,4820.029516862704,8826.573054901955,1867.8467677456115,2976.706801112528,5281.548704068176,6023.102525726356,3715.617042278906,3967.928994977252,4797.25911474523,624.3945855968032,7231.259392035446,3340.970928092926,7851.407816108862,4811.3678535195795,4959.268941957999,6799.9783243844695,7528.718259963739,11284.371959582386,7933.195588043383,6689.617925695694,10649.260020455089,7176.29925261